Пробная работа с SQLite3 по базе данных студентов "Академии "Калашников"

In [1]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [2]:
path = "Akademy_Kalashnikov_BD.sqlite"
connection = create_connection(path)

Connection to SQLite DB successful


Создам вспомогательную функцию, которая позволяет создавать таблицы и добавлять в них записи 

In [3]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

Создам и наполню записями две связанные таблицы - список групп и список обучающихся. 

In [4]:
create_groups_table = """
CREATE TABLE IF NOT EXISTS groups (
  code_group_name TEXT PRIMARY KEY,
  level TEXT NOT NULL,
  course_name TEXT
  );
"""
execute_query(connection, create_groups_table)

Query executed successfully


In [5]:
create_students_table = """
CREATE TABLE IF NOT EXISTS students (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  surname TEXT NOT NULL,
  code_group_name TEXT NOT NULL,
  FOREIGN KEY (code_group_name) REFERENCES groups (code_group_name)
 );
"""
execute_query(connection, create_students_table)

Query executed successfully


In [6]:
create_groups = """
INSERT INTO
    groups (code_group_name, level, course_name)
VALUES
    ('НР-11','Вводный','Нейро'),
    ('НР-12','Вводный','Нейро'),
    ('НР-21','Базовый','Нейро'),
    ('НР-31','Углубленный','Нейро'),
    ('РТ-11','Вводный','Робо'),
    ('РТ-12','Вводный','Робо'),
    ('РТ-32','Углубленный','Робо'),
    ('ИТ-11','Вводный','ИТ'),
    ('ИТ-21','Базовый','ИТ'),
    ('КС-11','Вводный','Космос');

"""
execute_query(connection, create_groups)

Query executed successfully


In [7]:
create_students = """
INSERT INTO
    students (name, surname, code_group_name)
VALUES
    ('Иван', 'Тургенев', 'ИТ-21'),
    ('Антон', 'Чехов', 'ИТ-11'),
    ('Федор', 'Достоевский', 'НР-31'),
    ('Александр', 'Пушкин', 'РТ-32'),
    ('Михаил', 'Булгаков', 'НР-31');
 
"""

execute_query(connection, create_students) 

Query executed successfully


Для извлечение данных из таблиц создам вспомогательную функцию

In [8]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

Извлеку все данные таблиц students и groups

In [9]:
select_students = "SELECT * from students"
students = execute_read_query(connection, select_students)

for student in students:
    print(student)

(1, 'Иван', 'Тургенев', 'ИТ-21')
(2, 'Антон', 'Чехов', 'ИТ-11')
(3, 'Федор', 'Достоевский', 'НР-31')
(4, 'Александр', 'Пушкин', 'РТ-32')
(5, 'Михаил', 'Булгаков', 'НР-31')


In [10]:
select_groups = "SELECT * from groups"
groups = execute_read_query(connection, select_groups)

for group in groups:
    print(group)

('НР-11', 'Вводный', 'Нейро')
('НР-12', 'Вводный', 'Нейро')
('НР-21', 'Базовый', 'Нейро')
('НР-31', 'Углубленный', 'Нейро')
('РТ-11', 'Вводный', 'Робо')
('РТ-12', 'Вводный', 'Робо')
('РТ-32', 'Углубленный', 'Робо')
('ИТ-11', 'Вводный', 'ИТ')
('ИТ-21', 'Базовый', 'ИТ')
('КС-11', 'Вводный', 'Космос')


Извлеку фамилии обучающихся, их группы, курсы и направления

In [11]:
select_students_groups = """
SELECT
    students.id,
    students.surname,
    groups.code_group_name,
    groups.level,
    groups.course_name
FROM
    groups
    INNER JOIN students ON students.code_group_name = groups.code_group_name

"""
students_groups = execute_read_query(connection, select_students_groups)

for students_group in students_groups:
    print(students_group)


(1, 'Тургенев', 'ИТ-21', 'Базовый', 'ИТ')
(2, 'Чехов', 'ИТ-11', 'Вводный', 'ИТ')
(3, 'Достоевский', 'НР-31', 'Углубленный', 'Нейро')
(4, 'Пушкин', 'РТ-32', 'Углубленный', 'Робо')
(5, 'Булгаков', 'НР-31', 'Углубленный', 'Нейро')


Подсчитаю, сколько учеников учится в каждой группе

In [12]:
select_students_groups_count = """
SELECT
    groups.code_group_name as Post,
    COUNT (students.id) as Students
FROM
    students,
    groups
WHERE
    groups.code_group_name = students.code_group_name
GROUP BY
    groups.code_group_name

"""

students_groups_count = execute_read_query(connection, select_students_groups_count)

for students_groups_ in students_groups_count:
    print(students_groups_)


('ИТ-11', 1)
('ИТ-21', 1)
('НР-31', 2)
('РТ-32', 1)


Изменю значение фамилии в таблице students

In [13]:
select_student_surname = """
SELECT surname FROM students WHERE id = 5
"""

student_surname = execute_read_query(connection, select_student_surname)

for surname in student_surname:
    print(surname)

('Булгаков',)


In [14]:
update_student_surname = """
UPDATE
    students
SET
    surname = "Лермонтов"
WHERE
    id = 5
"""

execute_query(connection, update_student_surname)

Query executed successfully


In [15]:
select_student_surname = """
SELECT surname FROM students WHERE id = 5
"""

student_surname = execute_read_query(connection, select_student_surname)

for surname in student_surname:
    print(surname)

('Лермонтов',)


Удалю все группы, относящиеся к направлению "Космос"

In [16]:
delete_group = "DELETE FROM groups WHERE groups.course_name ='Космос' "
execute_query(connection, delete_group)

Query executed successfully


In [17]:
select_groups = "SELECT * from groups"
groups = execute_read_query(connection, select_groups)

for group in groups:
    print(group)

('НР-11', 'Вводный', 'Нейро')
('НР-12', 'Вводный', 'Нейро')
('НР-21', 'Базовый', 'Нейро')
('НР-31', 'Углубленный', 'Нейро')
('РТ-11', 'Вводный', 'Робо')
('РТ-12', 'Вводный', 'Робо')
('РТ-32', 'Углубленный', 'Робо')
('ИТ-11', 'Вводный', 'ИТ')
('ИТ-21', 'Базовый', 'ИТ')


In [18]:
connection.close()